In [59]:
import recordlinkage
from recordlinkage.datasets import load_febrl4

# load example datasets
dfA, dfB = load_febrl4()

In [ ]:
import pandas as pd

In [66]:
dfA.head(2)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625


In [62]:
dfB.head(2)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


In [67]:
# define indexer
indexer = recordlinkage.Index()
indexer.full() 
candidate_links = indexer.index(dfA, dfB)

In [ ]:
# define comparison 
compare_cl = recordlinkage.Compare()
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')

In [68]:
# compute comparisons of candidate matches
features = compare_cl.compute(candidate_links, dfA, dfB)
features

surname  date_of_birth
rec_id_1     rec_id_2                              
rec-1070-org rec-561-dup-0       0.0              0
             rec-2642-dup-0      0.0              0
             rec-608-dup-0       0.0              0
             rec-3239-dup-0      0.0              0
             rec-2886-dup-0      0.0              0
...                              ...            ...
rec-66-org   rec-4495-dup-0      0.0              0
             rec-4211-dup-0      0.0              0
             rec-3131-dup-0      0.0              0
             rec-3815-dup-0      0.0              0
             rec-493-dup-0       0.0              0

[25000000 rows x 2 columns]

In [64]:
matches = features[features.sum(axis=1) == 2]
matches

,,surname,date_of_birth
rec_id_1,rec_id_2,,
rec-1016-org,rec-1016-dup-0,1.0,1
rec-4405-org,rec-4405-dup-0,1.0,1
rec-1288-org,rec-1288-dup-0,1.0,1
rec-3585-org,rec-3585-dup-0,1.0,1
rec-298-org,rec-298-dup-0,1.0,1
...,...,...,...
rec-3877-org,rec-3877-dup-0,1.0,1
rec-1622-org,rec-1622-dup-0,1.0,1
rec-2153-org,rec-2153-dup-0,1.0,1


In [65]:
# get and order on left and right index pairs
matches.reset_index(inplace=True)
left_records = dfA.reindex(matches['rec_id_1']).reset_index()
right_records = dfB.reindex(matches['rec_id_2']).reset_index()

# merge tables 
dfMerged = pd.concat([left_records, right_records], axis=1)
dfMerged.head()

,rec_id_1,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,...,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
0,rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,...,courtney,painter,12,pinkerton circuit,NaN,richlands,4560,vci,19161214,4066625
1,rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,...,charles,green,38,salkauskas crescent,klea,dapto,4566,nsw,19480930,4365168
2,rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,...,vanessa,parr,905,macquoid place,broadbridge manor,south qrafton,2135,sa,19951119,9239102
3,rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,...,mikayla,malloney,37,randwick road,avalsnd,hoppers crossing,4552,vic,19860208,7207688
4,rec-298-org,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,...,blake,howie,1,belmont park belted galloway stud,cutlack street,budgewoi,6071,vic,19250301,5180548
